<a href="https://colab.research.google.com/github/MariMurotani/ColabNotebooks/blob/main/DereflectFormerTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat May 24 06:36:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DereflectFormerTraining

/content/drive/MyDrive/Colab Notebooks/DereflectFormerTraining


In [ ]:
# ✅ Clone the repository
!git clone https://github.com/Agent76ow/DereflectFormer.git

fatal: destination path 'DereflectFormer' already exists and is not an empty directory.


In [ ]:
%cd DereflectFormer

/content/drive/MyDrive/Colab Notebooks/DereflectFormerTraining/DereflectFormer


In [ ]:
!pip install -r requirements.txt

Reason for being yanked: deprecated, use 4.5.5.64
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.7 MB/s e

In [ ]:
# configs/small_train_experimentを作成
!mkdir -p configs/small_train_experiment

# configs/indoor/default.json をコピー
!cp configs/indoor/default.json configs/small_train_experiment/default.json

In [ ]:
!pip uninstall -y opencv-python opencv-python-headless numpy
!pip install numpy==1.24.4 opencv-python-headless==4.8.0.76

In [ ]:
!pip install thop

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DereflectFormerTraining/DereflectFormer

In [ ]:
%cd datasets/
!git clone https://github.com/fqnchina/CEILNet.git
%cd ../

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DereflectFormerTraining/DereflectFormer

In [ ]:
# testdata_reflection_synthetic_table2を変換
!mkdir -p datasets/REFLECT/
!mkdir -p datasets/REFLECT/train
!mkdir -p datasets/REFLECT/train/reflect
!mkdir -p datasets/REFLECT/train/GT

In [ ]:
#
# CEILNetのデータを書き換え
#
import os
import shutil

input_dir = 'datasets/CEILNet/testdata_reflection_synthetic_table2'
output_reflect_dir = 'datasets/REFLECT/train/reflect'
output_gt_dir = 'datasets/REFLECT/train/GT'

os.makedirs(output_reflect_dir, exist_ok=True)
os.makedirs(output_gt_dir, exist_ok=True)

# ディレクトリ内のすべてのファイルから `-label1.png` に一致するものだけ抽出
for filename in os.listdir(input_dir):
    if filename.endswith('-label1.png'):
        base = filename.replace('-label1.png', '')  # 例: 2007_003506

        # 対応する -input.png ファイル名
        input_filename = f"{base}-input.png"
        input_path = os.path.join(input_dir, input_filename)
        gt_path = os.path.join(input_dir, filename)

        # 両方存在する場合のみコピー
        if os.path.exists(input_path):
            dst_input = os.path.join(output_reflect_dir, f"{base}.png")
            dst_gt = os.path.join(output_gt_dir, f"{base}.png")
            shutil.copy(input_path, dst_input)
            shutil.copy(gt_path, dst_gt)

In [ ]:
#
# テスト用のデータセット
#
src_reflect = 'datasets/REFLECT/train/reflect'
src_gt = 'datasets/REFLECT/train/GT'
dst_reflect = 'datasets/REFLECT/test/reflect'
dst_gt = 'datasets/REFLECT/test/GT'

os.makedirs(dst_reflect, exist_ok=True)
os.makedirs(dst_gt, exist_ok=True)

# 両方に存在するファイルのみコピー（最大10件）
count = 0
for fname in sorted(os.listdir(src_reflect)):
    if not os.path.exists(os.path.join(src_gt, fname)):
        continue  # 対応するGTがなければスキップ

    shutil.copy(os.path.join(src_reflect, fname), os.path.join(dst_reflect, fname))
    shutil.copy(os.path.join(src_gt, fname), os.path.join(dst_gt, fname))

    count += 1
    if count >= 10:
        break

In [ ]:
import os
import shutil

sir2_root = 'SIR2+'
output_reflect = 'datasets/REFLECT/train/reflect'
output_gt = 'datasets/REFLECT/train/GT'
os.makedirs(output_reflect, exist_ok=True)
os.makedirs(output_gt, exist_ok=True)

idx = 0
for root, dirs, files in os.walk(sir2_root):
    print(root)
    if 'mixture.png' in files and 'groundtruth.png' in files:
        mixture_path = os.path.join(root, 'mixture.png')
        gt_path = os.path.join(root, 'groundtruth.png')

        new_name = f"sir2_{idx:05d}.png"
        shutil.copy(mixture_path, os.path.join(output_reflect, new_name))
        shutil.copy(gt_path, os.path.join(output_gt, new_name))
        idx += 1

print(f"✅ 合計 {idx} 枚のペアを datasets/REFLECT/train にコピーしました")


✅ 合計 0 枚のペアを datasets/REFLECT/train にコピーしました


In [ ]:
!python train.py

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/drive/MyDrive/Colab Notebooks/DereflectFormerTraining/DereflectFormer/train.py:161: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested ma

In [ ]:
import os
import torch
from PIL import Image
import torchvision.transforms as T
from models import dereflectformer_b
from utils import hwc_to_chw, chw_to_hwc
import cv2
import numpy as np

# 入出力パス
image_path = 'test_data/balcony2_cube.jpg'  # 任意のテスト画像
output_path = 'output/balcony_result.png'
weights_path = 'saved_models/reflect/dereflectformer-b.pth'

# モデル定義
model = dereflectformer_b()
model = model.cuda()
model.eval()

# 学習済みの state_dict をロード
checkpoint = torch.load(weights_path)

# 'module.' を除去して state_dict を変換
state_dict = checkpoint['state_dict']
new_state_dict = {}
for k, v in state_dict.items():
    new_key = k.replace('module.', '')  # ここがポイント
    new_state_dict[new_key] = v

model.load_state_dict(new_state_dict)
# 画像読み込み・前処理
img = cv2.imread(image_path)[:, :, ::-1]  # BGR → RGB
img = img.astype(np.float32) / 255.0
img = (img - 0.5) * 2  # [0,1] → [-1,1]
img_tensor = torch.from_numpy(hwc_to_chw(img)).unsqueeze(0).cuda()

# 推論
with torch.no_grad():
    pred = model(img_tensor).clamp(-1, 1)
    pred = pred.squeeze().cpu().numpy()
    pred = (chw_to_hwc(pred) + 1) / 2  # [-1,1] → [0,1]
    pred = (pred * 255).astype(np.uint8)

# 保存
os.makedirs(os.path.dirname(output_path), exist_ok=True)
cv2.imwrite(output_path, pred[:, :, ::-1])  # RGB → BGR
print(f"✅ Saved to {output_path}")


✅ Saved to output/balcony_result.png


In [ ]:
####　データセットを増やして学習